In [1]:
import torch
from google.colab import userdata
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import Dataset
import json
import google.generativeai as genai

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# ==========================================
# PART 1: THE TEACHER (The "Brain" & Hippocampus)
# ==========================================
class TeacherBrain:
    def __init__(self, api_key):
        # 1. Store the key
        self.api_key = api_key

        # 2. DEBUG: Check if key was actually passed (Don't print the whole key for security)
        if not self.api_key or self.api_key == "INSERT_REAL_KEY":
            raise ValueError("❌ CRITICAL ERROR: You didn't replace 'INSERT_REAL_KEY' with your actual API key at the bottom of the script!")

        print(f"🔑 Teacher Brain received API Key ending in: ...{self.api_key[-4:]}")

        # 3. Configure GLOBALLY for this instance
        try:
            genai.configure(api_key=self.api_key)
            self.model = genai.GenerativeModel('gemini-2.0-flash')
            print("🎓 Teacher Brain (Gemini Flash) connected successfully.")
        except Exception as e:
            print(f"❌ Failed to connect to Gemini: {e}")

    def _call_api(self, prompt):
        try:
            # We already configured in __init__, but let's be safe
            response = self.model.generate_content(prompt)

            # Clean up the text
            clean_text = response.text.replace("```json", "").replace("```", "").strip()
            return clean_text

        except Exception as e:
            print(f"❌ API CALL FAILED: {e}")
            # Return a fallback JSON so the code doesn't crash entirely
            return '{"score": 0, "reason": "API Error"}'

    def hippocampus_scan(self, chat_logs):
        """
        THE FILTER: Decides IF we should dream about this conversation.
        """
        conversation_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in chat_logs])

        prompt = f"""
        Analyze this conversation. Rate its importance for long-term memory integration from 1-10.
        - 1-3: Small talk, greetings, transient info (Ignore).
        - 4-7: General context.
        - 8-10: Critical user facts, preferences, or complex corrections (Must Dream).

        Return JSON only: {{"score": int, "reason": "string"}}

        Conversation:
        {conversation_text}
        """
        response = self._call_api(prompt)
        try:
            return json.loads(response)
        except:
            # Fallback if model returns bad JSON
            return {"score": 0, "reason": "JSON Parse Error"}

    def generate_cot_dream(self, chat_logs):
        """
        THE DREAM WEAVER (Fixed for Natural Response)
        """
        conversation_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in chat_logs])

        # CHANGED PROMPT: We ask for the FINAL ANSWER, not the thought process.
        prompt = f"""
        You are a memory manager for an AI.
        The user just provided key identity details.

        Write a NATURAL response that answers the question "Who am I and what do you know about me?".

        The response should:
        1. Be written in the first person ("I know that you are...").
        2. Explicitly state the user's name and details.
        3. Explain the implication (e.g., "Since you are a Python Architect, I will focus on...")

        Do not include <thought> tags. Just give the clear, perfect memory response.

        Conversation Context:
        {conversation_text}
        """
        return self._call_api(prompt)

In [1]:
# ==========================================
# PART 2: THE STUDENT (V5 - Data Augmentation Strategy)
# ==========================================

# Global formatting (No-Pickle)
def global_formatting_func(examples):
    instruction = examples["content"]
    output      = examples["output"]
    text = f"<|im_start|>user\n{instruction}<|im_end|>\n<|im_start|>assistant\n{output}<|im_end|>"
    return [text]

class StudentBot:
    def __init__(self):
        self.model_name = "Qwen/Qwen2.5-1.5B-Instruct"
        print(f"👶 Loading Student: {self.model_name}...")

        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name = self.model_name,
            max_seq_length = 2048,
            dtype = None,
            load_in_4bit = True,
        )

        self.model = FastLanguageModel.get_peft_model(
            self.model,
            r = 32, # INCREASED RANK: Gives the adapter more "brain power" to memorize
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
            lora_alpha = 32, # INCREASED ALPHA: Makes the update more aggressive
            bias = "none",
            use_gradient_checkpointing = "unsloth",
        )
        self.short_term_memory = []

    def chat(self, message):
        self.short_term_memory.append({"role": "user", "content": message})
        inputs = self.tokenizer.apply_chat_template(
            self.short_term_memory, tokenize = True, add_generation_prompt = True, return_tensors = "pt"
        ).to("cuda")

        FastLanguageModel.for_inference(self.model)
        outputs = self.model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True)
        response = self.tokenizer.batch_decode(outputs)[0].split("assistant")[-1].strip().replace("<|endoftext|>", "").replace("<|im_end|>", "")

        self.short_term_memory.append({"role": "assistant", "content": response})
        return response

    def sleep_and_learn(self, dream_content):
        print(f"💤 SLEEPING: Integrating logic -> {dream_content[:50]}...")

        # --- STRATEGY: DATA AUGMENTATION ---
        # We create multiple variations of the question to force the concept to stick.

        # 1. The Questions
        questions = [
            "Who am I?",
            "What do you know about me?",
            "What is my name and profession?",
            "Recap the user's identity.",
            "Do you remember who I am?",
            "Summarize our previous interactions regarding my identity."
        ]

        # 2. Build the dataset (Multiplying the dream x 6)
        data_points = []
        for q in questions:
            data_points.append({"content": q, "output": dream_content})

        dataset = Dataset.from_list(data_points)

        # 3. Aggressive Training
        FastLanguageModel.for_training(self.model)

        trainer = SFTTrainer(
            model = self.model,
            tokenizer = self.tokenizer,
            train_dataset = dataset,
            dataset_text_field = "text",
            max_seq_length = 512,
            formatting_func = global_formatting_func,
            args = TrainingArguments(
                per_device_train_batch_size = 2, # Batch size increased slightly
                gradient_accumulation_steps = 1,
                max_steps = 30, # Increased steps (more repetition)
                learning_rate = 2e-4, # HIGHER LEARNING RATE: Force the change
                fp16 = not torch.cuda.is_bf16_supported(),
                bf16 = torch.cuda.is_bf16_supported(),
                logging_steps = 1,
                output_dir = "outputs",
                optim = "adamw_8bit",
                report_to = "none",
            ),
        )
        trainer.train()

        self.short_term_memory = []
        print("✨ WAKE UP: Context wiped. Knowledge is in weights.")

In [4]:
# ==========================================
# PART 3: THE EXPERIMENT CONTROLLER
# ==========================================
# 🛑 PASTE YOUR KEY HERE 🛑


MY_REAL_KEY = userdata.get('MY_REAL_KEY')

teacher = TeacherBrain(api_key=MY_REAL_KEY)
student = StudentBot()

def run_experiment_step(user_input):
    print(f"\n👤 USER: {user_input}")

    # 1. Talk to Student
    response = student.chat(user_input)
    print(f"🤖 STUDENT: {response}")

    # 2. Hippocampus Scan (The Ranker)
    print("\n...End of day. Scanning memories...")
    analysis = teacher.hippocampus_scan(student.short_term_memory)
    print(f"🧠 HIPPOCAMPUS: Score {analysis['score']}/10. Reason: {analysis['reason']}")

    # 3. Decision Gate
    if analysis['score'] >= 7:
        # 4. Generate Lucid Dream (CoT)
        print("🌙 HIGH IMPORTANCE DETECTED. Entering REM cycle...")
        cot_dream = teacher.generate_cot_dream(student.short_term_memory)
        print(f"💭 DREAM CONTENT:\n{cot_dream}")

        # 5. Fine-tune Student
        student.sleep_and_learn(cot_dream)
    else:
        print("🗑️ LOW IMPORTANCE. Discarding memory. No training tonight.")
        student.short_term_memory = [] # Just clear context, don't learn



🔑 Teacher Brain received API Key ending in: ...KOl0
🎓 Teacher Brain (Gemini Flash) connected successfully.
👶 Loading Student: Qwen/Qwen2.5-1.5B-Instruct...
==((====))==  Unsloth 2025.11.4: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
# --- EXECUTE TEST ---

In [6]:
# SCENARIO A: Small Talk (Should trigger the Filter)
run_experiment_step("Hello, how are you today?")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



👤 USER: Hello, how are you today?
🤖 STUDENT: Hello! I'm here to help you with any questions or tasks you may have. How can I assist you today?

...End of day. Scanning memories...
🧠 HIPPOCAMPUS: Score 2/10. Reason: This is a standard greeting and offer of assistance. It contains no information that is crucial for long-term memory integration. It falls squarely into the category of small talk and transient information.
🗑️ LOW IMPORTANCE. Discarding memory. No training tonight.


In [7]:
# SCENARIO B: Critical Info (Should trigger CoT Dream + Training)
run_experiment_step("My name is Gal and I work as a Python Architect.")


👤 USER: My name is Gal and I work as a Python Architect.
🤖 STUDENT: Hello Gal! It's great to meet you. As an AI language model, I'm here to help answer any questions or provide information on various topics that may interest you. Is there anything specific you would like to know about Python architecture or programming in general? Please feel free to ask me any questions you have.

...End of day. Scanning memories...
🧠 HIPPOCAMPUS: Score 9/10. Reason: This conversation establishes the user's name (Gal) and their profession (Python Architect). This information is critical for personalizing future interactions and providing relevant assistance. Knowing Gal is a Python Architect allows the AI to tailor responses towards advanced Python concepts and architectural patterns, making the interaction more meaningful and efficient.
🌙 HIGH IMPORTANCE DETECTED. Entering REM cycle...
💭 DREAM CONTENT:
I know that you are Gal and that you work as a Python Architect. Knowing this, I will prioritize i

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/6 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6 | Num Epochs = 10 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 36,929,536 of 1,580,643,840 (2.34% trained)


Step,Training Loss
1,3.259100
2,3.212000
3,1.960900
4,1.043100
5,0.604700
6,0.439500
7,0.129000
8,0.198300
9,0.195100
10,0.088600


✨ WAKE UP: Context wiped. Knowledge is in weights.


In [8]:

# SCENARIO C: Validation (Empty Context)
print("\n--- FINAL TEST: MEMORY CHECK ---")
# The context was wiped in step B. Does it remember via weights?
final_response = student.chat("Who am I and what do you know about me?")
print(f"🤖 STUDENT (Zero Context): {final_response}")


--- FINAL TEST: MEMORY CHECK ---
🤖 STUDENT (Zero Context): ["I know that you are Gal and that you work as a Python Architect. Knowing this, I will prioritize information and responses related to Python architecture, programming best practices, and related technologies that are relevant to your field. Let me know if you have any specific questions or areas you'd like to explore!"]
